In [17]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama

In [18]:
import requests
from bs4 import BeautifulSoup

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = "No body content found"


In [20]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [21]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [22]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [23]:
def summarize(url):
    website = Website(url)
    messages = messages_for(website)
    response = ollama.chat(model=MODEL, messages=messages)
    return response['message']['content']

In [24]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [26]:
display_summary("https://cnn.com")

**Summary of CNN Website**

### Overview
The website is a news and information platform run by the American cable news channel CNN.

### Featured Sections
- **Breaking News**: The latest news headlines, including updates on global events, politics, and entertainment.
- **Analysis**: In-depth analysis of current events, business, technology, and more.
- **Top Stories**: A curated list of the most important stories from around the world.
- **Video**: Videos covering a range of topics, including news, analysis, interviews, and documentaries.

### News Announcements
The website is currently reporting on several key events, including:
- The Gaza ceasefire
- Developments in Ukraine
- US politics, particularly with regards to former President Trump's actions

### Other Features
- **Editorials**: Opinion pieces on various topics.
- **Features**: Longer articles exploring specific subjects or issues.
- **Podcasts**: A range of audio programs covering news, analysis, and entertainment.

### Technical Information
- The website has experienced technical issues with video players.